In [1]:
%load_ext autoreload
%autoreload 2

import kyklos as ky
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'
import heyoka as hy

In [2]:
# build EOMs for basic 2-body system
x, y, z, vx, vy, vz = hy.make_vars("x", "y", "z", "vx", "vy", "vz")
r = hy.sqrt(x**2 + y**2 + z**2)
mu=3.986004418e5
# Assemble system
sys = [
    (x, vx),
    (y, vy),
    (z, vz),
    (vx, -mu * x / r**3),
    (vy, -mu * y / r**3),
    (vz, -mu * z / r**3)
]
state_kep = ky.OE([7500,0,0,0,0,0],'kep',mu=mu)
state_cart = ky.OE.to_cartesian(state_kep)

t_end = 2*np.pi*np.sqrt(7500**3/mu)
grid = np.linspace(0,t_end,1000)
ta = hy.taylor_adaptive(sys,[0.0] * 6)
ta

C++ datatype            : double
Tolerance               : 2.220446049250313e-16
High accuracy           : false
Compact mode            : false
Taylor order            : 20
Dimension               : 6
Time                    : 0
State                   : [0, 0, 0, 0, 0, 0]

In [3]:
# ta.state[:] = state_cart.elements
# traj = ta.propagate_grid(grid)[5]
# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=traj[:,0], 
#     y=traj[:,1],
#     mode='lines',
#     name='Trajectory',
#     line=dict(color='blue', width=2)
# ))
# fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio

# fig.show()

In [4]:
# ta.time = 0.0
# ta.state[:] = state_cart.elements
# c_out = ta.propagate_until(t_end, c_output=True)[4]
# traj_c = c_out(grid)

# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=traj_c[:,0], 
#     y=traj_c[:,1],
#     mode='lines',
#     name='Trajectory',
#     line=dict(color='red', width=2)
# ))
# fig.update_yaxes(scaleanchor="x", scaleratio=1)  # Equal aspect ratio

# fig.show()

In [5]:
nan_state = np.array([7000, 0, 0, np.nan, 0, 7.5])
inf_state = np.array([7000, np.inf, 0, 0, 0, 7.5])
zero_state = np.array([0, 0, 0, 1, 1, 1])
ta.time = 0.0
ta.state[:] = zero_state
c_out = ta.propagate_until(t_end, c_output=True)[4]
ta, c_out
# traj_c = c_out(grid)
# c_out

(C++ datatype            : double
 Tolerance               : 2.220446049250313e-16
 High accuracy           : false
 Compact mode            : false
 Taylor order            : 20
 Dimension               : 6
 Time                    : nan
 State                   : [nan, nan, nan, nan, nan, nan],
 None)

In [8]:
ta.time = 0.0
ta.state[:] = state_cart.elements
c_out = ta.propagate_until(t_end, c_output=True)[4]
traj_c = c_out(grid)
c_out(t_end)

array([ 7.50000000e+03,  3.76341940e-11,  0.00000000e+00, -3.67642326e-14,
        7.29018008e+00,  0.00000000e+00])

In [11]:
c_out(100.0)

array([ 7.46459674e+03,  7.27870552e+02,  0.00000000e+00, -7.07507653e-01,
        7.25576726e+00,  0.00000000e+00])